# Podstawy programowania (AD) 2

## Tomasz Rodak

Wykład V

## Przestrzeń nazw (*namespace*)

**Przestrzeń nazw** to przyporządkowanie od nazw do obiektów. Z jednej strony mamy obiekty, z drugiej ich nazwy i zbiór tych nazw tworzy przestrzeń nazw. 

Języki programowania zwykle dopuszczają równoczesne istnienie wielu rozłącznych przestrzeni nazw. Dzięki temu:
* łatwiej unikać kolizji między nazwami;
* funkcje mogą występować w roli "czarnych skrzynek" nie wpływających na stan środowiska.

Język Python, tak jak wiele innych języków programowania, korzysta z wielu przestrzeni nazw. Ze względu na rodzaj w kierunku malejącej ogólności są to:
1. wbudowana przestrzeń nazw,
2. globalna przestrzeń nazw,
3. zakres dołączony (omówimy na następnym wykładzie),
4. lokalna przestrzeń nazw.

Interpreter, widząc odniesienie do nazwy, przeszukuje te przestrzenie w kierunku od najmniej do najbardziej ogólnej. Zasada ta nosi nazwę [LEGB](https://sebastianraschka.com/Articles/2014_python_scope_and_namespaces.html#3-legb---local-enclosed-global-built-in).

### Wbudowana przestrzeń nazw

Wbudowana przestrzeń nazw (*built-in namespace*) grupuje nazwy obiektów wbudowanych i stanowiących rdzeń języka Python. Obiekty wbudowane są dostępne poprzez swoje nazwy zawsze i bez konieczności wykonywania żadnego importu. 
Przykłady obiektów wbudowanych:
* funkcje `print()`, `int()`, ([dokumentacja](https://docs.python.org/3/library/functions.html#built-in-funcs))
* stałe `True`, `None`, ([dokumentacja](https://docs.python.org/3/library/constants.html#built-in-consts))
* wyjątki `TypeError`, `SyntaxError`, ([dokumentacja](https://docs.python.org/3/library/exceptions.html)).

W Pythonie przestrzeń nazw implementowana jest jako słownik. Z tego powodu instrukcja przypisania wykorzystująca istniejącą nazwę likwiduje możliwość dostępu do pierwotnego obiektu. 

Przykład nieprawidłowego nazewnictwa zmiennych:

```python
>>> ## Szukamy liczby z zakresu 1..1000 o największej 
>>> ## sumie dzielników właściwych.
>>> a, b = 1, 1000 
>>> liczby = range(a, b+1)
>>> max = 0 # Przesłonięcie wbudowanej funkcji max()
>>> for n in liczby:
...     sum = 0 # Przesłonięcie wbudowanej funkcji sum()
...     for d in range(1, n//2 + 1):
...         if n % d == 0:
...             sum += d
...     if sum > max:
...         max = sum
...         liczba = n
...
>>> print(f'Liczba: {liczba}\nSuma dzielników właściwych: {max}')
Liczba: 960
Suma dzielników właściwych: 2088
>>> sum([1, 2, 3])
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-34-5b30f1e835bd> in <module>
----> 1 sum([1, 2, 3])

TypeError: 'int' object is not callable
>>> max([3, -1, 0])
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-35-acfd7b805aec> in <module>
----> 1 max([3, -1, 0])

TypeError: 'int' object is not callable
```

Problemem w tym programie jest niewłaściwe wykorzystanie nazw: wbudowane funkcje `sum()` i `max()` zostały przesłonięte przez zmienne o tych samych nazwach. W wyniku tego nie można już korzystać z tych funkcji.

Aby przywrócić nazwom oryginalne znaczenie wystarczy wykorzystać moduł [`builtins`](https://docs.python.org/3/library/builtins.html), który zawiera identyfikatory obiektów wbudowanych:

```python
>>> import builtins
>>> sum = builtins.sum
>>> max = builtins.max
>>> sum([1, 2, 3])
6
>>> max([3, -1, 0])
3
```

**_Ćwiczenie._** *Popraw nazwy zmiennych w omawianym wyżej programie. Dobierz je tak, aby nie przesłaniały zmiennych wbudowanych ale zachowały czytelność.*


Niektóre ciągi znaków są zastrzeżone i nie można używać ich jako nazw. Nazywamy je **słowami kluczowymi** (*keywords*). Lista wszystkich słów kluczowych znajduje się w module [`keyword`](https://docs.python.org/3/library/keyword.html):

```python
>>> import keyword
>>> for k in keyword.kwlist:
...     print(k, end=' ')
False None True and as assert async await break class continue def del elif else except finally for from global if import in is lambda nonlocal not or pass raise return try while with yield 
```

Zwróć uwagę, że wśród słów kluczowych tylko `True`, `False` i `None` są nazwami rozumianymi jako nazwy obiektów. Pozostałe stanowią składnię języka. Słowa `if`, `for` czy `return` nie są nazwami żadnych obiektów (i nie mogą być), więc nie są elementami wbudowanej (ani żadnej innej) przestrzeni nazw.

### Globalna przestrzeń nazw. Moduły

W Pythonie modułem jest każdy plik z kodem w języku Python. Modułami są również tzw. moduły rozszerzeń, czyli programy napisane w C, C++, Javie itp. tak, aby były możliwe do zwykłego zaimportowania. 

**Globalna przestrzeń nazw** (albo **zasięg globalny**) to przestrzeń nazw definiowana przez moduł (lub pakiet, czyli folder modułów i pakietów). 

Moduły definiują swoje przestrzenie nazw za pomocą instrukcji przypisania. Pamiętaj jednak, że w Pythonie instrukcja przypisania ma wiele form: poza znakiem `=` również instrukcje `def`, `class` czy `import` tworzą przypisanie.

Globalne przestrzenie nazw są rozłączne a ich komunikację zapewnia instrukcja `import`. Każda globalna przestrzeń nazw sama ma nazwę. Sprawdzamy ją odczytując wartość zmiennej specjalnej `__name__`.

Globalna przestrzeń nazw dostępna w sesji interaktywnej, w komórce notatnika Jupyter, czy w skrypcie wykonywanym z linii komend jest szczytowym środowiskiem uruchomieniowym kodu. Ten specjalny zasięg globalny ma nazwę `__main__` (zobacz [Top-level code environment](https://docs.python.org/3/library/__main__.html) w dokumentacji).

Oto test w sesji interaktywnej uruchomionej np. w programie IDLE:
```python
>>> __name__
'__main__'
```
Podobny test w komórce notatnika Jupyter:

In [1]:
__name__

'__main__'

Nazwy zdefiniowane w module zobaczysz wywołując w tym module funkcję `dir()`. Zwracana wartość zależy od modułu, w którym funkcja została wywołana oraz od tego jakie nazwy aż do momentu jej wywołania zostały zdefiniowane. Oto wynik w dopiero co otwartej konsoli Pythona:
```
Python 3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0] :: Anaconda, Inc. on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> dir()
['__annotations__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__']
```
Zauważ, że nie ma tu funkcji `sum()`, `print()` czy stałej `True`. One nie należą do przestrzeni nazw modułu `__main__`.

Nazwy zdefiniowane w skrypcie wykonywanym z poziomu wiersza poleceń, lub przez IDE takie jak IDLE czy Thonny, wchodzą automatycznie w skład przestrzeni nazw modułu `__main__`. Aby to sprawdzić utwórz w edytorze program `p1.py` o zawartości

```python
print(f'Moduł: {__name__}')
```

Następnie w terminalu w katalogu, w którym się ten program znajduje, wykonaj:
```
$ python p1.py
```
Zobaczysz napis 
```
Moduł: __main__
```
Ten sam efekt będzie miało wykonanie tego programu z poziomu np. IDLE czy Thonny, czyli poprzez wciśnięcie przycisku `Run`.

Podsumowując, `__main__` jest nazwą globalnej przestrzeni nazw stanowiącej [szczytowe środowisko uruchomieniowe](https://docs.python.org/3/library/__main__.html#what-is-the-top-level-code-environment) języka Python. Możesz myśleć, że jest po prostu kolejna globalna przestrzeń nazw. Pochodzi ona z modułu [`__main__`](https://docs.python.org/3/library/__main__.html) z biblioteki standardowej a jej wyjątkowość polega na tym, że musi zostać utworzona po to, aby w ogóle cokolwiek mogło działać.

Instrukcja `import` jest formą instrukcji przypisania. Jeśli moduł importowany jest za pomocą składni
```python
import <nazwa_modułu>
```
lub
```python
import <nazwa_modułu> as <nazwa>
```
to w tej (globalnej) przestrzeni nazw, która wykonuje import, tworzona jest tylko jedna nowa zmienna globalna `<nazwa_modułu>` (ewentualnie `<nazwa>`). Wówczas dostęp do nazw z zaimportowanego modułu zapewnia dobrze już nam znana *notacja z kropką*:

```python
>>> import itertools
>>> list(itertools.permutations('abc'))
[('a', 'b', 'c'), ('a', 'c', 'b'), ('b', 'a', 'c'), ('b', 'c', 'a'), ('c', 'a', 'b'), ('c', 'b', 'a')]
```

Tym co odróżnia rodzaje przestrzeni nazw jest porządek w jakim interpreter je przeszukuje (LEGB) oraz czas ich życia. Globalna przestrzeń nazw modułu zaczyna być dostępna po uruchomieniu go jako programu głównego (wtedy jest przestrzeń `__main__`) lub po wykonaniu importu i żyje aż do końca działania skryptu. Jeśli jest to przestrzeń `__main__`, to istnieje i jest dostępna przez cały czas działania skryptu. Zmienne, do których wykonane jest odwołanie w globalnej przestrzeni nazw są poszukiwane najpierw w tej przestrzeni, a dopiero potem w przestrzeni nazw obiektów wbudowanych.

Co się dzieje, gdy z poziomu `__main__` odwołujesz się do np. do funkcji `sum()`?

W module `__main__` takiej nazwy nie ma, a tam właśnie interpreter będzie szukał najpierw. Ponieważ spotka go niepowodzenie, więc przeniesie się na wyższy poziom ogólności, czyli poszuka nazwy `sum` w przestrzeni nazw obiektów wbudowanych. Tam ta nazwa faktycznie się znajduje i wskazuje na funkcję wbudowaną obliczającą sumę.

A co się dzieje z wbudowaną funkcją `sum()`, gdy wykonamy coś takiego:

```python
>>> sum = 123321
>>> sum
123321
```

Odwołanie do `sum` zmusza interpreter do rozpoczęcia poszukiwań nazwy, najpierw w module gdzie odwołanie zostało wykonane, czyli w naszym przypadku w `__main__`. Ponieważ już tutaj ta nazwa zostaje znaleziona, więc dalszych poszukiwań interpreter nie prowadzi -- zmienna `sum` jest liczbą całkowitą 123321. Nazwa `sum` nie została przeniesiona z funkcji na liczbę całkowitą, została raczej **przesłonięta**.

Ten sam efekt wystąpi, gdy wewnątrz jakiegoś modułu napiszesz:
```python
# modul.py
print(sum) # Wbudowana funkcja sum()
sum = 123321
print(sum) # Liczba całkowita 123321, nazwa sum przesłonięta    
```

### Tworzenie modułów. Importowanie

Tworzenie modułów w j. Python jest bardzo proste, gdyż po prostu każdy skrypt Pythona z programem jest równocześnie modułem. 

Utwórz w dowolnym edytorze skrypt z programem:

```python
# Path: src/module_demo.py
a = 'Ala ma kota.'

print(f'Moduł: {__name__}')

def f():
	print(a)

f()
```
Nazwij go `module_demo.py` i zapisz w katalogu bieżącym. Potraktujemy ten program jak moduł i zaimportujemy go do środowiska. Aby moduł ten został znaleziony, katalogiem bieżącym interpretera musi być katalog zawierający ten właśnie moduł. Zmianę katalogu bieżącego wykonasz funkcją `os.chdir()`.  

```python
>>> import os
>>> os.chdir('/tmp/') ## Wpisz swoją ścieżkę.
>>> import module_demo
Moduł: module_demo
Ala ma kota.
>>> module_demo
<module 'module_demo' from '/tmp/module_demo.py'>
>>> module_demo.a
'Ala ma kota.'
>>> a = "Monty Python"
>>> module_demo.f
<function f at 0x7f99238b1710>
>>> module_demo.f()
Ala ma kota.
```

Widzimy, że:
* Moduł został wykonany jak zwykły program.
* Wewnątrz modułu nazwa globalnej przestrzeni nazw to `module_demo`. 
* Nazwa modułu została wprowadzona do modułu `__main__` jako zmienna globalna.
* Notacja z kropką pozwala na dostęp do zmiennych i funkcji z modułu.
* Zmienne zdefiniowane w module są dostępne w module, ale nie w module `__main__` (nie popadają w konflikt z nazwami zdefiniowanymi w module `__main__`).

Podsumowując, jeśli dane są dwa moduły `A` i `B` to definiowane przez nie globalne przestrzenie nazw są rozłączne. Moduł `B` może importować moduł `A` składnią
```python
import A
```
Wówczas w przestrzeni definiowanej przez `B` pojawia się nazwa `A` zapewniająca dostęp do obiektów `A` za pomocą notacji z kropką. Podczas tak wykonywanego importu cały kod znajdujący się w `A` zostaje wykonany. 

Ostatnia uwaga stwarza problem, gdy skrypt gra równoczesnie rolę wykonalnego programu i modułu z obiektami wartymi importowania. Pojawia się wtedy konieczność zabezpieczenia kodu, który nie powinien być wykonany w przypadku importu. W tym celu stosuje się konstrukcję warunkową:

```python
<Kod modułu>

if __name__ == '__main__':
    <Kod programu>
```

Jeśli moduł jest wykonywany jako program, to `__name__` przyjmuje wartość `__main__` i kod wewnątrz warunku zostaje wykonany. W przeciwnym przypadku, gdy moduł jest importowany, kod wewnątrz warunku nie jest wykonywany, gdyż `__name__` przyjmuje wtedy nazwę modułu.

Wspomnijmy jeszcze o dwóch innych sposobach importowania:
```python
from <nazwa_modułu> import <nazwa>
```
oraz
```python
from <nazwa_modułu> import <nazwa> as <inna_nazwa>
```
W tym przypadku z modułu importowany jest jedynie obiekt `<nazwa>`, przy czym w drugim przypadku będzie on funkcjonował jako `<inna_nazwa>`. Reszta modułu jak i sam moduł pozostają niedostępne. Również w tym przypadku **cały** moduł zostaje odczytany, wykonany i przetłumaczony na kod bajtowy.

Zwykle za złą praktykę uważa się pobieranie z modułu wszystkich nazw jak leci:
```python
from <nazwa_modułu> import *
```
Ten kod importuje z modułu wszystkie nazwy z wyjątkiem tych zaczynających się podkreślnikiem. Wadą tego rozwiązania jest to, że miesza ono przestrzenie nazw. Ponadto jeśli moduł jest duży, to i nowych nazw będzie bardzo dużo. Tym niemniej czasem się tak postępuje, przede wszystkim w sesji interaktywnej, przykład znajdziesz [tutaj](https://docs.sympy.org/latest/tutorial/intro.html#the-power-of-symbolic-computation).

Więcej informacji o modułach znajdziesz [tym rozdziale](https://docs.python.org/3/tutorial/modules.html#) oficjalnego tutorialu.

### Lokalna przestrzeń nazw

Lokalne przestrzenie nazw tworzone są podczas rozmaitych okazji, przede wszystkim podczas wywoływania funkcji i metod. Nazwa utworzona w takiej przestrzeni nazywana jest *lokalną*. Nazwy utworzona w przestrzeni globalnej, to nazwy *globalne*, no i mamy jeszcze omówione wyżej nazwy *wbudowane*. Nazwy globalne i wbudowane nietrudno odróżnić, gdyż wbudowane należą do rdzenia języka, a globalne tworzone są na poziomie modułu. Jak jednak odróżnić nazwy lokalne od globalnych? W jaki sposób nazwy lokalne powstają?

W dalszej dyskusji ograniczę się do lokalnych przestrzeni nazw tworzonych przez wywołania funkcji. Przestrzenie lokalne tworzone w innych kontekstach nie będą się istotnie różniły. 

Oto przykład ilustrujący istnienie lokalnej przestrzeni nazw. Wykonaj go w serwisie PythonTutor.

```python
s = 'Jestem Ziemianinem!'

def f():
    s = 'Jestem Marsjaninem!'
    print(s)

f()
print(s)
```

Rezultat:
```
Jestem Marsjaninem!
Jestem Ziemianinem!
```

Dlaczego widzimy "Jestem Ziemianinem!", pomimo tego że łańcuch `s` został zmieniony na "Jestem Marsjaninem!" i to dwa razy: w ciele funkcji i podczas jej wywołania? Otóż przypisanie 
```python
s = 'Jestem Ziemianinem!'
```
tworzy zmienną globalną `s`. Fakt, że w ciele funkcji `f` mamy przypisanie
```python
s = 'Jestem Marsjaninem!'
```
oznacza, że w lokalnej przestrzeni nazw utworzonej przez wywołanie `f()` tworzona jest nowa nazwa lokalna `s`. Przestrzeń globalna i lokalna są rozłączne. Dlatego nazwa `s` z przestrzeni globalnej nie została przeniesiona na nowy łańcuch. Przeciwnie, przez chwilę istnieją dwie różne nazwy `s` znajdujące się w różnych przestrzeniach. Gdy w wywołaniu `f()` interpreter dociera do `print(s)`, to widzi odwołanie do zmiennej `s` i zaczyna jej poszukiwanie w bieżącej przestrzeni lokalnej. Ponieważ tam zmienną znajduje, więc jego poszukiwania na tym się kończą. Okazuje się, że uzyskany wyżej efekt jest wypadkową istnienia różnych przestrzeni nazw i hierarchii w jakiej interpreter je przeszukuje. Gdy funkcja zakończy działanie cała lokalna przestrzeń nazw utworzona z okazji jej wywołania zostaje zapomniana. Znakomicie ilustrują to rysunki w serwisie PythonTutor. `print(s)` z ostatniej linii odwołuje się do zmiennej globalnej `s`, która nie została zmodyfikowana przez wcześniejsze wywołanie `f()`, a jedynie na chwilę przesłonięta.

```python
s = 'Jestem Ziemianinem!'

def f():
    print(s)

f()
print(s)
```
Wynik:
```
Jestem Ziemianinem!
Jestem Ziemianinem!
```

Ten przykład jest już teraz jasny. Podczas wywołania `f()` tworzona jest lokalna przestrzeń nazw, ale nie ma w niej zmiennej `s`. Dlatego interpreter przenosi poszukiwania do przestrzeni globalnej. Gdyby i tam nie znalazł, to przeszedłby do nazw wbudowanych i wtedy w razie niepowodzenia zgłosił `NameError`.

Oto mniej oczywisty przykład:

```python
s = 'Jestem Ziemianinem!'

def f():
    print(s) 
    s = 'Jestem Marsjaninem!'
    print(s)

f()
print(s)
```

Wynik:
```
---------------------------------------------------------------------------
UnboundLocalError                         Traceback (most recent call last)
<ipython-input-25-f0269b267812> in <module>
      6     print(s)
      7 
----> 8 f()
      9 print(s)

<ipython-input-25-f0269b267812> in f()
      2 
      3 def f():
----> 4     print(s)
      5     s = 'Jestem Marsjaninem!'
      6     print(s)

UnboundLocalError: local variable 's' referenced before assignment
```

Dostajemy dziwny wyjatek `UnboundLocalError` choć na pierwszy rzut wydaje się, że pownniśmy zobaczyć:
```
Jestem Ziemianinem!
Jestem Marsjaninem!
Jestem Ziemianinem!
```
Jednak wybór projektowy zastosowany w języku jest inny. Ponieważ zmienna `s` jest przypisana w ciele funkcji, więc podczas wywołania `f()` tworzona jest zmienna lokalna `s` i interpreter wie o tym **zanim dotrze do przypisania**
```python
s = 'Jestem Marsjaninem!'
```
Faktyczne przypisanie wykonywane jest później, wtedy gdy interpreter dotrze do wiersza z przypisaniem, jednak zbiór nazw z tworzonej lokalnej przestrzeni nazw znany jest natychmiast. Ponieważ `s` jest lokalna i nie została jeszcze przypisana, więc nie może zostać wyświetlona, i w ogóle nie może się powieść żadne do niej odwołanie. Rzucony wyjątek jest przypadkiem szczególnym `NameError`, o czym przekonuje nas [hierarchia wyjątków](https://docs.python.org/3/library/exceptions.html#exception-hierarchy).

#### Instrukcja `global`

Deklaracja `global` informuje o tym, że zmienna przypisana w ciele funkcji ma być traktowana jak globalna. 

```python
s = 'Jestem Ziemianinem!'

def f():
    global s
    print(s)
    s = 'Jestem Marsjaninem!'
    print(s)

f()
print(s) # s została zmieniona 
```

Wynik:
```
Jestem Ziemianinem!
Jestem Marsjaninem!
Jestem Marsjaninem!
```

Zmienna `s` jest przypisana w ciele `f`, ale jest też zadeklarowana jako globalna. Dlatego w lokalnej przestrzeni nazw tworzonej przez wywołanie `f()` nazwy `s` nie ma. Nazwa `s` istnieje tylko w zakresie globalnym. Z tego też powodu przypisanie

```python
s = 'Jestem Marsjaninem!'
```
przenosi nazwę `s` z jednego łańcucha na drugi. Łańcuch `'Jestem Ziemianinem!'` zostaje zapomniany.

Tego typu zmiana nosi w programowaniu nazwę *efektu ubocznego* (<a href="https://en.wikipedia.org/wiki/Side_effect_(computer_science)"><em>side effect</em></a>). Chodzi o to, że stan środowiska przed wywołaniem funkcji i po jej wywołaniu nie jest taki sam -- w naszym przypadku zmienna `s` zmieniła wartość. 

Kod, który wprowadza wiele efektów ubocznych może początkowo wydawać się prosty w pisaniu, w rzeczywistości jednak szybko się komplikuje i staje trudny do odczytania, testów i dalszego rozwoju.